## Lyrics Hypothesis Test

Testing an assumption that a person would perform better than a robot in a rap battle, I have constructed a data-oriented challenge for myself. 

The challenge here is to explore whether a rap purely written by myself, with help of rhyming words, or a completely autogenerated model. I'm curious to understand if the model would be improved by other artist lyrics in the genre. As the song itself has a tag by the name of an artist is trying to emulate, I will look for the artist it has identified as well as other rappers similarly associated with these artists. I will build three models of this type as the second stage of the project. I engage in this work through segmenting the rappers by gender, if defined. If I have to categorize the genders individually, I will explore difference based on a scaCy sentiment model. Based on the resulting percentage, it would be classified as positive negative and neutral. This will provide a larger scoped project as the hypothesis now includes many variables, but I will be performing all songs generated, so the results will only show the variance of the song lyrics.

Other implementations are found online, but they are simpler in comparison, don't offer the source code, or the tools they were using are now obsolete. In this case, I will be using Tensorflow and spaCy to qualify the text data and build a model that hopefully captures the complexity of the song.

One area of interest during the model building stage will be there be challenges with modeling lyrics to a beat captured from YouTube that was labeled free. This may present new challenges. I will also explore literature on if there is a modeling technique that encorporates the music notes of the corresponding music for each some. As this is a similar musical project, I will be placing some constraints on the scope of the project.

Lyric genuis: https://lyricsgenius.readthedocs.io/en/master/

In [26]:
import lyricsgenius
import pytorch_pretrained_bert
import torchtext
import time
import json
from pandas.io.json import json_normalize
import pandas as pd
import os
import plotly.express as px
import re

In [ ]:
genius = lyricsgenius.Genius()

In [ ]:
def retrieve_lyrics_from_list(artist_names:list):
    for artist in artist_names:
        lyrics = genius.search_artist(artist, max_songs=50, sort='popularity', 
                                per_page=50, get_full_info=True)
        lyrics.save_lyrics(filename=f'{artist}.json',
                    overwrite=True,verbose=True)
        time.sleep(20)
    return "Lyrics available in the current working directory."

female_artists = [
    'Cardi B', 'Nicki minaj', 'Megan thee stallion', 'CupcakKe', 'BIA', 
    'Lakeyah', 'Latto', 'Doja cat', 'Tierra Whack', 'City girls'
                  ]
male_artists = [
    'Gucci mane', 'Pooh shiesty', 'G herbo', 'Wale', 'Big sean', 
    'Logic', 'J. Cole', 'Tee Grizzley', 'Da Baby', 'Moneybagg Yo'
                ]

retrieve_lyrics_from_list(female_artists)
retrieve_lyrics_from_list(male_artists)

Searching for songs by Tee Grizzley...

Song 1: "First Day Out"
Song 2: "From The D to the A"
Song 3: "No Effort"
Song 4: "Young Grizzley World"
Song 5: "Jetski Grizzley"
Song 6: "Satish"
Song 7: "2 Vaults"
Song 8: "Real Niggas"
Song 9: "Win"
Song 10: "Colors"
Song 11: "Beef"
Song 12: "Fuck It Off"
Song 13: "Straight to It"
Song 14: "Country"
Song 15: "How Many"
Song 16: "Second Day Out"
Song 17: "God’s Warrior"
Song 18: "First Day Out (Remix)"
Song 19: "Secrets"
Song 20: "Don’t Even Trip"
Song 21: "I Apologize"
Song 22: "Side Nigga"
Song 23: "Pray for the Drip"
Song 24: "Activated"
Song 25: "10K"
Song 26: "Sweet Thangs"
Song 27: "Lost and Found"
Song 28: "Day Ones"
Song 29: "Had To"
Song 30: "No Talkin"
Song 31: "Bloodas 2 Interlude"
Song 32: "The Smartest Intro"
Song 33: "Robbery"
Song 34: "Light"
Song 35: "Payroll"
Song 36: "Grizzley Gang"
Song 37: "Overlapped"
Song 38: "Trenches"
Song 39: "My Moment (Intro)"
Song 40: "Too Lit"
Song 41: "Catch It"
Song 42: "Still My Moment"
Song 43:

'Lyrics available in the current working directory.'

## Exploratory data analysis

Using only the Latto dataset to estimate any of the features that are available in 

In [3]:
def create_dataframe(json_filepath):
    with open(json_filepath) as json_data:
        data = json.load(json_data)

    df = pd.DataFrame(data['songs'])
    selected=['annotation_count','title','artist','artist_names',
              'pyongs_count', 'stats','release_date', 'lyrics']
    result = df[selected].join(pd.json_normalize(df[selected]['stats'])[['hot','pageviews']])

    return result

def join_json_dir(directory):
    result = None

    for filename in os.listdir(directory):
        if filename[-4:] == 'json':
            f = os.path.join(directory, filename)
            if result is None:
                result = create_dataframe(filename)
            else:
                result = pd.concat([result,create_dataframe(filename)], ignore_index=True)
    return result

lyrics_data = join_json_dir('/work/Machine-Learning/Lyric Creation')

Now that the lyrics are extracted and combined we can explore the lyrics as well as other statistics. Additional data can be added to give a holistic picture. For the following curious questions sections, we will look at the lyrics data in isolation.

### Curious questions are as follows:
1. What's the distribution of songs per artist?
2. What's the distribution of word counts per song?
3. How many unique words exist in this vocabulary?
4. Which artist has the most expansive vocabulary?
5. ...

In [ ]:
title_count = lyrics_data.groupby('artist')['title'].count()
px.bar(title_count)

In [28]:
for i in (lyrics_data['lyrics'][30]).split('['):
    spl = i.split(']')
    if len(spl) > 1:
        print(spl[0],':')
        print(spl[1][:-6])

Intro :

I was hungry as hell writing this shit
I don't even give no fuck
Let'
Hook :

Cinnamon toast crunch, cinna cinnamon toast crunch
Cinnamon toast crunch, cinna cinnamon toast crunch (Yo! Yo!)
I got bitches wanna box me like I'm cinnamon toast crunch
I just pull up and I square up like I'm cinnamon toast crunch
Cinnamon toast crunch, cinna cinnamon toast crunch
Cinnamon toast crunch, cinna cinnamon toast crunch
I got bitches wanna box me like I'm cinnamon toast crunch
I just pull up and I square up like I'm cinnamon toast c
Verse 1 :

It ain't no glitch when your heart stop
I've been that bitch since the start, Doc
Dude wanna trap me, that smart cock
But I made him pull out like a park lot
I'm counting real bank in a Rover
At the most, gave me cold shoulders
But now I'm making bitches pay for it
Like a new fit at FashionNova
You could never do it, watch how I pursue it
Money, I've been doing, I need life insurance
Bitch yo money little, so we call you Stuart
Been there, done that

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=24d50205-0d01-4bd6-a263-9bbd6567f7de' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>